In [1]:
import re
import numpy as np

import findspark
import os
findspark.init()

import pyspark
sc = pyspark.SparkContext()


In [3]:
# this code will look for term through all pages of newspaper instead of only the front page 
# to search only the front page, need better tag parsing using lxml
# data for this is on VM ubuntu@206.12.96.6
# to access the notebook that is running already, set up tunnel
# ssh  ubuntu@206.12.96.6 -L 9995:localhost:9995
# then go to http://localhost:9995/tree in your browser

def process_string(whole_newspaper):
    list_of_lines=whole_newspaper.split("\n")
    regEx = re.compile(r"<wd pos=\"(\d+,\d+,\d+,\d+)\">([\w\'\",\[\]\{\}\(\)\.]+)</wd>")
    
    list_positions=[]
    for line in list_of_lines:
        line=line.strip()
        m = re.search(regEx,line)
        if m !=None :
            if m.group(2).lower() == searchTerm:
                list_positions.append(m.group(1))
    return list_positions
    
def see_set_content(something):
    return process_string(something[1]) #something[0] is the name of file, something[1] is the content

searchTerm = "war"
searchTerm = searchTerm.lower()
    
frontPage = np.zeros((7000,5250)) 
frontPage.dtype
frontPage = frontPage.astype('uint32')

# use wild cards to select range of dates
wholefiles = sc.wholeTextFiles('hdfs://hdp:9000/ocrdata/*1944*FEB10*')
wholefiles.cache()

output = wholefiles.map(see_set_content)

for l in output.collect():
    for j in l:
        pos = str(j).split(',')
        posList = [int(item) for item in pos]
        frontPage[posList[1]:posList[3],posList[0]:posList[2]] += 1
        
import matplotlib.pyplot as plt
% matplotlib inline
plt.figure(figsize=(200, 200))
image  = plt.imshow(frontPage)
plt.show()

In [46]:
# this code will look for term only on the front page of the newspaper
# data for this is on VM ubuntu@206.12.96.6
# to access the notebook that is running already, set up tunnel
# ssh  ubuntu@206.12.96.6 -L 9995:localhost:9995
# then go to http://localhost:9995/tree in your browser

from bs4 import BeautifulSoup

def process_string_front_only(whole_newspaper):
    
    soup = BeautifulSoup(whole_newspaper,"lxml")
    taglist = soup.find_all("text")
    
    list_positions=[]
    for tagtxt in taglist: 
        pageinfo=tagtxt.find_all("pg")
        pagenumber=pageinfo[0]["pgref"]
        if pagenumber == "1":
            words=tagtxt.find_all("wd")
            for w in words:
                if(w.string.lower() == searchTerm):
                    list_positions.append(w["pos"])
                
    return list_positions
    
def see_set_content(something):
    return process_string_front_only(something[1]) #something[0] is the name of file, something[1] is the content


searchTerm = "war"
searchTerm = searchTerm.lower()
frontPage = np.zeros((7000,5250)) 
frontPage.dtype
frontPage = frontPage.astype('uint32')

# use wild cards to select range of dates
wholefiles = sc.wholeTextFiles('hdfs://hdp:9000/ocrdata/*1944*FEB10*')
wholefiles.cache()

output = wholefiles.map(see_set_content)
#print(output.collect())
for l in output.collect():
    for j in l:
        pos = str(j).split(',')
        posList = [int(item) for item in pos]
        frontPage[posList[1]:posList[3],posList[0]:posList[2]] += 1
        
import matplotlib.pyplot as plt
% matplotlib inline
plt.figure(figsize=(200, 200))
image  = plt.imshow(frontPage)
plt.show()

[['1961,3938,2024,3963', '4360,2479,4420,2503', '1137,2879,1201,2907', '1096,1245,1175,1273', '1946,3165,2025,3193', '1535,6507,1615,6535'], ['542,2028,621,2057', '1168,4740,1247,4770', '2274,1782,2339,1811', '2556,3717,2620,3744', '4760,659,4856,688', '4743,5102,4808,5130'], ['1708,812,1790,842', '1662,1847,1766,1880', '1502,2028,1586,2058', '1591,3164,1654,3194', '2408,2080,2486,2108', '3697,1348,3765,1376', '3621,4854,3685,4882', '4044,5024,4111,5053', '4101,5416,4166,5445'], ['2323,3181,2401,3209', '2122,6206,2183,6235', '2689,6576,2749,6608', '4270,1746,4336,1773'], ['1045,5554,1147,5583', '751,6390,814,6417', '4097,5972,4161,6001', '1754,2615,1857,2644', '2522,1296,2601,1325', '2471,1384,2552,1413'], ['1372,5218,1435,5246', '757,6127,816,6155', '1315,6134,1382,6161', '1874,1223,1940,1251', '1632,2211,1736,2243', '2218,5808,2300,5836'], ['679,1573,749,1602', '1329,3812,1392,3842', '1276,6407,1340,6434', '1761,1219,1825,1247', '2361,758,2462,787', '2225,778,2305,806', '2520,965,258